https://huggingface.co/stabilityai/stable-diffusion-2-1

In [ ]:
pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from datetime import datetime
import random
import sys
from pathlib import Path
from IPython.display import Image, display

model_version = "2-1"
model_id = f"stabilityai/stable-diffusion-{model_version}"
print(model_id)
    
for i in range(sys.maxsize):
    # Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")
    pipe.enable_attention_slicing()
    
    seed = random.randint(0, sys.maxsize)

    generator = torch.Generator("cuda").manual_seed(seed)
    
    prompt = None

    #
    # Settings
    #
    # START
    #
    
    # Prompts moved to prompts.txt
    
    # Default: 768 × 768 - Has to be divisible by 8
    width = 1216
    height = 760
    # Values that works great (without running out of memory 😥)
    # 960x960: 921 600 pixels
    # 1216x760: 924 160 pixels
    
    # Default: 50
    num_inference_steps = 175
    
    # Default: 7.5 - "Values between 7 and 8.5 are usually good choices for Stable Diffusion"
    guidance_scale = 7.75
    
    #
    # Settings
    #
    # END
    #
    
    prompt = random.choice(list(open('prompts.txt'))).rstrip()

    image = pipe(prompt, generator=generator, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps, height=height, width=width).images[0]

    save_dir = Path("stable-diffusion-art/{}".format(prompt.replace(" ", "_")))
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # datetime object containing current date and time
    now = datetime.now()
    
    save_path = "{}/{}_{}x{}_i{}_g{}_s{}_v{}.png".format(str(save_dir), prompt.replace(" ", "_"), width, height, num_inference_steps, guidance_scale, seed, model_version) 
    
    image.save(save_path)
    
    display(Image(filename=save_path))